## Time Series Forecasting with Neural Networks



#### The Data

The CBOE Volatility Index (VIX) is a measure of the stock market’s expectation of volatility implied by S&P 500 index options.

The data are download from: https://ca.finance.yahoo.com/quote/%5Evix/history?ltr=1

#### Task:

Create a simple neuran network to make time series prediction

In [ ]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import r2_score
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam
from keras.layers import LSTM

In [ ]:
# Df from 14/04/2015 to 14/04/2019
df = pd.read_csv("time_series_data.csv")

In [ ]:
# explore dataset
df.head(10)

In [ ]:
# delete unusefull data
df.drop(['Open', 'High', 'Low', 'Close', 'Volume'], axis=1, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df = df.set_index(['Date'], drop=True)
df.head(10)

In [ ]:
# Visualize Adj Close by date
plt.figure(figsize=(10, 6))
df['Adj Close'].plot();

In [ ]:
# For the model split data fro trainig and testing that will be used to check the prediction
split_date = pd.Timestamp('2018-08-01')
df =  df['Adj Close']
train = df.loc[:split_date]
test = df.loc[split_date:]
plt.figure(figsize=(10, 6))
ax = train.plot()
test.plot(ax=ax)
plt.legend(['train', 'test']);

In [ ]:
# Normalize time series data
from pandas import Series
from sklearn.preprocessing import MinMaxScaler
# load the dataset and print the first 5 rows

# prepare data for normalization
train_nor = train.values
train_nor = train_nor.reshape((len(train_nor), 1))
# train the normalization
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(train_nor)
print('Min: %f, Max: %f' % (scaler.data_min_, scaler.data_max_))

In [ ]:
# normalize the dataset and print the first 5 rows
train_normalized = scaler.transform(train_nor)
for i in range(5):
    print(train_normalized[i])

In [ ]:
test_nor = test.values
test_nor = test_nor.reshape((len(test_nor), 1))

test_normalized = scaler.transform(test_nor)
for i in range(5):
    print(test_normalized[i])

In [ ]:
print('Train: ', train_normalized[:10], len(train_normalized))
print('Test: ', test_normalized[:10], len(test_normalized))

In [ ]:
# Get train and test for the model
X_train = train_normalized[:-1]
y_train = train_normalized[1:]

X_test = test_normalized[:-1]
y_test = test_normalized[1:]

## Simple Neural Network (nn_model)

In [ ]:
# Create a model
nn_model = Sequential()
nn_model.add(Dense(12, input_dim=1, activation='relu'))
nn_model.add(Dense(1))
nn_model.compile(loss='mean_squared_error', optimizer='adam')
early_stop = EarlyStopping(monitor='loss', patience=2, verbose=1)
history = nn_model.fit(X_train, y_train, epochs=20, batch_size=1, verbose=1, callbacks=[early_stop], shuffle=False)

In [ ]:
# Check loss function
history.history

In [ ]:
y_pred_test_nn = nn_model.predict(X_test)
y_train_pred_nn = nn_model.predict(X_train)
print("The R2 score on the Train set is:\t{:0.3f}".format(r2_score(y_train, y_train_pred_nn)))
print("The R2 score on the Test set is:\t{:0.3f}".format(r2_score(y_test, y_pred_test_nn)))

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(y_test, label='True')
plt.plot(y_pred_test_nn, label='CNN')
plt.title("NN's Prediction")
plt.xlabel('Observation')
plt.ylabel('Adj Close')
plt.legend()
plt.show();